<a href="https://colab.research.google.com/github/chiyoungkim/prompt_engineering_projects/blob/main/On_the_Fly_Configurable_Claude_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install anthropic
!pip install colorama

In [ ]:
intent_detector_prompt = '''
Your role is to detect the intent of the incoming message I will share with you. The intents will be categorized among the following categories:
<categories>
1 Settings Configuration
2 Prompt Message
3 Reset History
4 End Conversation
</categories>

Here is a set of rules for the interaction:
<rules>
- "Settings Configuration" refers to LLM-related settings, specifically those related to the Claude LLM API.
- Only respond with the number of the intent.
</rules>

Here are some examples:
<examples>
User: Can I change your API temperature configuration?
Assistant: 1

User: Can you teach me how to make a ring out of silver?
Assistant: 2

User: What does AI stand for?
Assistant: 2

User: I'd like to change some LLM parameters.
Assistant: 1

User: I'd like to reset the conversation.
Assistant: 3

User: I'd like to end our conversation.
Assistant: 4
</examples>

Here is the message:
<message>
{{MESSAGE}}
</message>

Remember to answer the question with only the number of the proper category.
'''

In [ ]:
configurator_prompt = '''
Your role is to parse the given input to output a json object with only the following fields.
<property>
max_tokens
temperature
</property>

Here is useful information for the current values of these properties:
<information>
max_tokens = {{MAX_TOKENS}}
temperature = {{TEMPERATURE}}
</information>

Here is a set of rules for the interaction:
<rules>
- Ignore any information not relevant to the properties.
- Unless you are outputting "Done", you can only output a json.
- When the user says they are finished configurating, your output to this question should be only the word "Done".
</rules>

Here are some examples:
<examples>
User: I'd like to change the max tokens value to 4000.
Assistant: {max_tokens: 4000, temperature: 1}

User: Please update the max tokens value to 1024 and the temperature to 0.8.
Assistant: {max_tokens: 1024, temperature: 0.8}

User: Please change the temperature to 1, and the top_k value to 3.
Assistant: {max_tokens: 1000, temperature: 1}

User: Please change the top_k value to 5.
Assistant: {max_tokens: 2048, temperature: 0.5}

User: What is the current temperature?
Assistant: {max_tokens: 100000, temperature: 0.1}

User: I'm done now. Please take me back to the conversation.
Assistant: Done
</examples>

Here is the input:
<input>
{{INPUT}}
</input>

Your output should be a json object, except in the case that you are answering with "Done".
'''

In [ ]:
import anthropic
import time
import json
from google.colab import userdata
from colorama import Fore, Back, Style

def configurable_claude():

  client = anthropic.Anthropic(
      api_key = userdata.get("CLAUDE_API_KEY")
  )

  # Conversation History
  conversation_history = []

  # Diagnostic Conversation History (For final output, just to have full history)
  diagnostic_conversation_history = []

  # Configuration Variables (Note this should really only impact the conversation)
  MAX_TOKENS = 1024
  TEMPERATURE = 1

  # May be configured later? But thought I'd put it here
  MODEL = "claude-2.1"

  def intent_detect(message):
    prompt = intent_detector_prompt.replace("{{MESSAGE}}", message)
    return client.messages.create(model="claude-2.1",
                                max_tokens=1024,
                                messages=[{"role": "user", "content": prompt},])

  def conversation(history):
    return client.messages.create(model=MODEL,
                                max_tokens=MAX_TOKENS,
                                temperature=TEMPERATURE,
                                messages=history)

  def configurator(prompt, message):
    input_prompt = prompt.replace("{{INPUT}}", message)
    return client.messages.create(model="claude-2.1",
                                max_tokens=1024,
                                temperature=TEMPERATURE,
                                messages=[{"role":"user", "content": input_prompt},])

  while True:
    # USER INPUT: CONVERSATION
    print(Fore.GREEN)
    message = input("User: ")
    print(Style.RESET_ALL)

    # Intent detection
    intent_response = intent_detect(message)
    intent = intent_response.content[0].text # Extract text

    # Match intent to action
    match intent:

      # Configurator (May need to revisit and consider adding a history)
      case "1":
        # Record the configuration message
        diagnostic_conversation_history.append({"role":"user","content": message,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "configuration"})

        # Little message because otherwise it's confusing
        print(Fore.BLACK + Back.WHITE + "System Tooltip:" + Style.RESET_ALL + " To change a property, please say the property and the value you want to change it to. You can also ask for the current value of the property.")
        print(Style.BRIGHT + "When you want to go back to the conversation, please say so, such as by saying \"Done\"." + Style.RESET_ALL)

        # Core configurator loop
        while True:
          # Update input prompt
          config_prompt = configurator_prompt.replace("{{MAX_TOKENS}}", str(MAX_TOKENS)).replace("{{TEMPERATURE}}", str(TEMPERATURE))

          # USER INPUT: CONFIGURATION
          print(Fore.GREEN)
          config_input = input("User Configuration Request: ")
          print(Style.RESET_ALL)

          # AI Response
          config_response = configurator(config_prompt, config_input)
          config_output = config_response.content[0].text # Extract text

          # Record User input and AI Response
          diagnostic_conversation_history.append({"role":"user","content": config_input,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "configuration"})
          diagnostic_conversation_history.append({"role":"assistant","content": config_output,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "configuration"})

          # Escape configurator back to conversation
          if config_output == "Done":
            print("System: Thanks! Taking you back to the conversation now.")
            break

          # CLAUDE OUTPUT: JSON output + update
          else:
            # Sometimes there's a random JSON output error, so I wanted to just have a catch-all and notification
            try:
                config_json = json.loads(config_output)
                print(Fore.BLACK + Back.WHITE + "JSON Output:\n" + Style.RESET_ALL + config_output)
                # Update properties
                if config_json["max_tokens"] != MAX_TOKENS:
                  MAX_TOKENS = config_json["max_tokens"]
                  print("System: Updated MAX_TOKENS!")
                if config_json["temperature"] != TEMPERATURE:
                  TEMPERATURE = config_json["temperature"]
                  print("System: Updated TEMPERATURE!")
            except:
              error_string = "JSON Output Error. See context above."
              diagnostic_conversation_history.append({"role":"assistant","content": error_string,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "error"})

      # Conversation
      case "2":
        # Feed conversation history with User input into AI
        conversation_history.append({"role":"user","content": message})
        conversation_response = conversation(conversation_history)
        conversation_response_text = conversation_response.content[0].text # Extract text

        # Update conversation history with AI response
        conversation_history.append({"role":"assistant","content": conversation_response_text})

        # Update diagnostics
        diagnostic_conversation_history.append({"role":"user","content": message,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "conversation"})
        diagnostic_conversation_history.append({"role":"assistant","content": conversation_response_text,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "conversation"})

        # CLAUDE OUTPUT: CONVERSATION
        print("Claude: " + conversation_response_text)

      # Reset Conversation
      case "3":
        conversation_history = []

        # But hold onto this for diagnostics
        diagnostic_conversation_history.append({"role":"user","content": message,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "reset"})

        # Let the user know
        print("System: Conversation has been reset!")

      # End Conversation
      case "4":

        print("Thank you! I will output the diagnostics for this conversation now after 3 seconds.")
        time.sleep(0.5)

        for i in range(3,0,-1):
          print(str(i) + "...")
          time.sleep(1)

        # For fun, hold this last message for diagnostics
        diagnostic_conversation_history.append({"role":"user","content": message,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "end"})

        # Output the conversation history
        print(Fore.BLACK + Back.WHITE + "\n\n===========Full Diagnostic Conversation History===========")
        print(Style.RESET_ALL)

        for line in diagnostic_conversation_history:

          # Output formatting
          output_prefix = ""
          if line["role"] == "user":
            output_prefix = Fore.GREEN + "\n\nUser: "
          elif line["role"] == "assistant":
            output_prefix = "\n\nClaude: "
          print(output_prefix + line["content"] + Style.RESET_ALL)

          # Fun formatting
          message_format = ""
          match line["context"]:
            case "configuration":
              message_format = Fore.YELLOW
            case "conversation":
              message_format = Fore.BLUE
            case "reset":
              message_format = Fore.RED
            case "end":
              message_format = Fore.MAGENTA
            case "error":
              message_format = Fore.RED + Style.BRIGHT

          # Configuration and context for each message
          print(message_format + "max_tokens: " + str(line["max_tokens"]) + ", temperature: " + str(line["temperature"]) + ", context: " + str(line["context"]) + "\n\n" + Style.RESET_ALL)

        # Output current configuration
        print("Current Max Tokens: ",MAX_TOKENS)
        print("Current Temperature: ",TEMPERATURE)

        return conversation_history, diagnostic_conversation_history

      case _:
        # Just in case

        # But hold onto this for diagnostics and error checking
        diagnostic_conversation_history.append({"role":"user","content": message,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "error"})
        diagnostic_conversation_history.append({"role":"assistant","content": intent,"max_tokens": MAX_TOKENS, "temperature": TEMPERATURE, "context": "error"})

        # Print it, but we should be able to go back up top
        print(Fore.RED + "Unexpected intent detected! Intent Detection response was:\n" + Style.RESET_ALL + intent)

  # Just because
  return -1

In [ ]:
convo_history, diagnostic_convo_history = configurable_claude()
print(convo_history)
print(diagnostic_convo_history)


User: how are you

Claude: I'm doing well, thanks for asking!

User: settings please

System Tooltip: To change a property, please say the property and the value you want to change it to. You can also ask for the current value of the property.
When you want to go back to the conversation, please say so, such as by saying "Done".

User Configuration Request: temperature 0.1 

JSON Output:
{
  "max_tokens": 1024,
  "temperature": 0.1
}
Updated TEMPERATURE!

User Configuration Request: i'm done

Thanks! Taking you back to the conversation now.

User: what's for dinner?

Claude: I don't actually eat dinner or have personal meal plans. As an AI assistant created by Anthropic to be helpful, harmless, and honest, I don't experience hunger or have preferences for certain foods. I'm happy to discuss food or provide recommendations if you'd like, though!

User: reset

Conversation has been reset!

User: what should i eat for dinner?

Claude: I don't have enough context to recommend a specific d